In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline

# Load data

In [2]:
DATA_FOLDER = '../readonly/final_project_data/'

sales    = pd.read_csv(os.path.join(DATA_FOLDER, 'sales_train.csv.gz'))
items           = pd.read_csv(os.path.join(DATA_FOLDER, 'items.csv'))
item_categories = pd.read_csv(os.path.join(DATA_FOLDER, 'item_categories.csv'))
shops           = pd.read_csv(os.path.join(DATA_FOLDER, 'shops.csv'))
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [3]:
sales.describe(percentiles=[0.03, 0.25, 0.5, 0.75, 0.97])

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,3.300173e+01,1.019723e+04,8.908532e+02,1.242641e+00
std,9.422988e+00,1.622697e+01,6.324297e+03,1.729800e+03,2.618834e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01
3%,0.000000e+00,4.000000e+00,1.307000e+03,7.900000e+01,1.000000e+00
25%,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00
50%,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00
75%,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00
97%,3.200000e+01,5.800000e+01,2.144000e+04,3.190000e+03,3.000000e+00
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03


# Aggregate data

In [4]:
from itertools import product
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols).agg({'item_cnt_day': np.sum, 'item_price': np.mean}).reset_index()

#fix column names
gb.rename(index=str,columns={'item_cnt_day': 'item_cnt_month'},inplace=True)

#join aggregated data to the grid
monthly_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)

#sort the data
monthly_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

In [5]:
monthly_data.head()

,shop_id,item_id,date_block_num,item_cnt_month,item_price
139255,0,19,0,0.0,0.0
141495,0,27,0,0.0,0.0
144968,0,28,0,0.0,0.0
142661,0,29,0,0.0,0.0
138947,0,32,0,6.0,221.0


In [6]:
monthly_data.describe(percentiles=[0.03, 0.97])

,shop_id,item_id,date_block_num,item_cnt_month,item_price
count,1.091385e+07,1.091385e+07,1.091385e+07,1.091385e+07,1.091385e+07
mean,3.118720e+01,1.130926e+04,1.497334e+01,3.342731e-01,1.166070e+02
std,1.734959e+01,6.209978e+03,9.495618e+00,3.417243e+00,6.584686e+02
min,0.000000e+00,0.000000e+00,0.000000e+00,-2.200000e+01,0.000000e+00
3%,3.000000e+00,9.870000e+02,0.000000e+00,0.000000e+00,0.000000e+00
50%,3.000000e+01,1.139100e+04,1.400000e+01,0.000000e+00,0.000000e+00
97%,5.800000e+01,2.154300e+04,3.200000e+01,2.000000e+00,9.990000e+02
max,5.900000e+01,2.216900e+04,3.300000e+01,2.253000e+03,3.079800e+05


# Combine train and test for lag features

In [7]:
test = pd.read_csv(os.path.join(DATA_FOLDER, 'test.csv.gz'))
test_date_block = monthly_data.date_block_num.max() + 1
test["date_block_num"] = test_date_block
monthly_data["ID"] = -1
all_data = monthly_data.append(test)
all_data.head()

,shop_id,item_id,date_block_num,item_cnt_month,item_price,ID
139255,0,19,0,0.0,0.0,-1
141495,0,27,0,0.0,0.0,-1
144968,0,28,0,0.0,0.0,-1
142661,0,29,0,0.0,0.0,-1
138947,0,32,0,6.0,221.0,-1


# Remove outliers

In [8]:
def clip_by_value(x, min_val, max_val):
    return min(max(min_val, x), max_val)

In [9]:
all_data["item_cnt_month"] = all_data["item_cnt_month"].map(lambda x: clip_by_value(x, 0, 20))
all_data["item_price"] = all_data["item_price"].map(lambda x: clip_by_value(x, 0, 10000))

# Add lag features

In [10]:
index_cols = ['shop_id', 'item_id', 'date_block_num']
cols_to_rename = ["item_cnt_month", "item_price"]

shift_range = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10 , 11, 12]
min_date_block = 12

lag_features = []

for month_shift in shift_range:
    all_data_shift = all_data[index_cols + cols_to_rename].copy()
    
    all_data_shift['date_block_num'] = all_data_shift['date_block_num'] + month_shift
    
    foo = lambda x: '{}_lag_{}'.format(x, month_shift) if x in cols_to_rename else x
    all_data_shift = all_data_shift.rename(columns=foo)

    lag_features += ['{}_lag_{}'.format(x, month_shift) for x in cols_to_rename]
    all_data = pd.merge(all_data, all_data_shift, on=index_cols, how='left').fillna(0)

del all_data_shift


In [11]:
print(lag_features)

['item_cnt_month_lag_1', 'item_price_lag_1', 'item_cnt_month_lag_2', 'item_price_lag_2', 'item_cnt_month_lag_3', 'item_price_lag_3', 'item_cnt_month_lag_4', 'item_price_lag_4', 'item_cnt_month_lag_5', 'item_price_lag_5', 'item_cnt_month_lag_6', 'item_price_lag_6', 'item_cnt_month_lag_7', 'item_price_lag_7', 'item_cnt_month_lag_8', 'item_price_lag_8', 'item_cnt_month_lag_9', 'item_price_lag_9', 'item_cnt_month_lag_10', 'item_price_lag_10', 'item_cnt_month_lag_11', 'item_price_lag_11', 'item_cnt_month_lag_12', 'item_price_lag_12']


# Split train and test data

In [12]:
train = all_data[all_data.date_block_num < test_date_block].copy()
del train["ID"]
test = all_data[all_data.date_block_num == test_date_block].copy()

In [13]:
train.head()

,shop_id,item_id,date_block_num,item_cnt_month,item_price,item_cnt_month_lag_1,item_price_lag_1,item_cnt_month_lag_2,item_price_lag_2,item_cnt_month_lag_3,...,item_cnt_month_lag_8,item_price_lag_8,item_cnt_month_lag_9,item_price_lag_9,item_cnt_month_lag_10,item_price_lag_10,item_cnt_month_lag_11,item_price_lag_11,item_cnt_month_lag_12,item_price_lag_12
0,0,19,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,27,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,28,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,29,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,32,0,6.0,221.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Add mean encoding

In [14]:
train = pd.merge(train, items[["item_id", "item_category_id"]], on=["item_id"], how="left")
test = pd.merge(test, items[["item_id", "item_category_id"]], on=["item_id"], how="left" )
train["shop_item_id"] = train["shop_id"].astype(str) + "-" + train["item_id"].astype(str)
test["shop_item_id"] = test["shop_id"].astype(str) + "-" + test["item_id"].astype(str)

In [15]:
target = "item_cnt_month"
alpha1 = 100
alpha2 = 10

target_mean = train[target].mean()
item_target_mean = train.groupby("item_id")[target].transform("mean")
item_target_count = train.groupby("item_id")[target].transform("count")
train["item_target_enc"] = (item_target_mean * item_target_count + target_mean * alpha1) / (item_target_count + alpha1)
train["item_target_enc"].fillna(target_mean, inplace=True)

shop_target_mean = train.groupby("shop_id")[target].transform("mean")
shop_target_count = train.groupby("shop_id")[target].transform("count")
train["shop_target_enc"] = (shop_target_mean * shop_target_count + target_mean * alpha1) / (shop_target_count + alpha1)
train["shop_target_enc"].fillna(target_mean, inplace=True)

cate_target_mean = train.groupby("item_category_id")[target].transform("mean")
cate_target_count = train.groupby("item_category_id")[target].transform("count")
train["cate_target_enc"] = (cate_target_mean * cate_target_count + target_mean * alpha1) / (cate_target_count + alpha1)
train["cate_target_enc"].fillna(target_mean, inplace=True)

shop_item_target_mean = train.groupby("shop_item_id")[target].transform("mean")
shop_item_target_count = train.groupby("shop_item_id")[target].transform("count")
train["shop_item_target_enc"] = (shop_item_target_mean * shop_item_target_count + target_mean * alpha2) / (shop_item_target_count + alpha2)
train["shop_item_target_enc"].fillna(target_mean, inplace=True)


In [16]:
train.head()

,shop_id,item_id,date_block_num,item_cnt_month,item_price,item_cnt_month_lag_1,item_price_lag_1,item_cnt_month_lag_2,item_price_lag_2,item_cnt_month_lag_3,...,item_cnt_month_lag_11,item_price_lag_11,item_cnt_month_lag_12,item_price_lag_12,item_category_id,shop_item_id,item_target_enc,shop_target_enc,cate_target_enc,shop_item_target_enc
0,0,19,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,40,0-19,0.212579,0.687348,0.243834,0.271127
1,0,27,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,19,0-27,0.085607,0.687348,0.701182,0.248533
2,0,28,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,30,0-28,0.163776,0.687348,1.077136,0.248533
3,0,29,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,23,0-29,0.099344,0.687348,0.638486,0.271127
4,0,32,0,6.0,221.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,40,0-32,1.223502,0.687348,0.243834,1.581867


In [17]:
item_target_enc_map = train.groupby("item_id")["item_target_enc"].first()
shop_target_enc_map = train.groupby("shop_id")["shop_target_enc"].first()
cate_target_enc_map = train.groupby("item_category_id")["cate_target_enc"].first()
shop_item_target_enc_map = train.groupby("shop_item_id")["shop_item_target_enc"].first()

test["item_target_enc"] = test["item_id"].map(item_target_enc_map)
test["item_target_enc"].fillna(target_mean, inplace=True)

test["shop_target_enc"] = test["shop_id"].map(shop_target_enc_map)
test["shop_target_enc"].fillna(target_mean, inplace=True)

test["cate_target_enc"] = test["item_category_id"].map(cate_target_enc_map)
test["cate_target_enc"].fillna(target_mean, inplace=True)

test["shop_item_target_enc"] = test["shop_item_id"].map(shop_item_target_enc_map)
test["shop_item_target_enc"].fillna(target_mean, inplace=True)

enc_features = ['item_target_enc', 'shop_target_enc', 'cate_target_enc', 'shop_item_target_enc']

In [18]:
test.head()

,shop_id,item_id,date_block_num,item_cnt_month,item_price,ID,item_cnt_month_lag_1,item_price_lag_1,item_cnt_month_lag_2,item_price_lag_2,...,item_cnt_month_lag_11,item_price_lag_11,item_cnt_month_lag_12,item_price_lag_12,item_category_id,shop_item_id,item_target_enc,shop_target_enc,cate_target_enc,shop_item_target_enc
0,5,5037,34,0.0,0.0,0,0.0,0.0,1.0,749.5,...,2.0,1999.0,1.0,2599.0,19,5-5037,1.729459,0.180471,0.701182,0.665933
1,5,5320,34,0.0,0.0,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,55,5-5320,0.298240,0.180471,0.223777,0.298240
2,5,5233,34,0.0,0.0,2,1.0,1199.0,3.0,999.0,...,0.0,0.0,0.0,0.0,19,5-5233,1.322227,0.180471,0.701182,0.763671
3,5,5232,34,0.0,0.0,3,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,23,5-5232,0.745956,0.180471,0.638486,0.306338
4,5,5268,34,0.0,0.0,4,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,20,5-5268,0.298240,0.180471,1.566275,0.298240


# Train/Validation split

In [19]:
valid_date_block  = test_date_block - 1
valid = train[train.date_block_num==valid_date_block].copy()
train = train[train.date_block_num < valid_date_block]
train = train[train.date_block_num >= min_date_block]

In [20]:
len(train), len(test)

(6186922, 214200)

In [21]:
features = lag_features + enc_features


# Linear regression

In [22]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(train[features])
X_valid_scaled = scaler.transform(valid[features])
X_test_scaled = scaler.transform(test[features])

In [23]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import SGDRegressor
sgd_reg = SGDRegressor(random_state=42, learning_rate="optimal", alpha=0.1)
X_scaled = np.concatenate([X_train_scaled, X_valid_scaled])
y = np.concatenate([train[target], valid[target]])
sgd_reg.fit(X_scaled, y)

SGDRegressor(alpha=0.1, average=False, early_stopping=False, epsilon=0.1,
             eta0=0.01, fit_intercept=True, l1_ratio=0.15,
             learning_rate='optimal', loss='squared_loss', max_iter=1000,
             n_iter_no_change=5, penalty='l2', power_t=0.25, random_state=42,
             shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [24]:
test["item_cnt_month"] = sgd_reg.predict(X_test_scaled)
test["item_cnt_month"].fillna(0, inplace=True)
test["item_cnt_month"] = test["item_cnt_month"].map(lambda x: min(max(0, x), 20))
test[["ID", "item_cnt_month"]].to_csv("submission2_sgd.csv", index=False)